# Create a notebook to perform RK4 integration with adaptive step sizes

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Define out derivatives

In [ ]:
def dydx(x,y):
    
    # set the derivatives
    
    # our equation is d^2y/dx^2 = -y
    
    # so we can write
    # dydx = z
    # dzdx = -y
    
    # we will set y = y[0]
    # we will set z = y[1]
    
    # declare an array
    y_derivs = np.zeros(2)
    
    # set dydx = z
    y_derivs[0] = y[1]
    
    # set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    # here we hav to return an array
    return y_derivs

# Define the 4th order RK method

In [ ]:
def ck_mv_core(dydx,xi,yi,nv,h):
    
    # declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    c2 = 1/5
    c3 = 3/10
    c4 = 3/5
    c5 = 1
    c6 = 7/8
    
    a21 = 1/5
    a31 = 3/40
    a32 = 9/40
    a41 = 3/10
    a42 = -9/10
    a43 = 6/5
    a51 = -11/54
    a52 = 5/2
    a53 = -70/27
    a54 = 35/27
    a61 = 1631/55296
    a62 = 175/512
    a63 = 574/13824
    a64 = 44275/110592
    a65 = 253/4096
    
    b1 = 37/378
    b2 = 0
    b3 = 250/621
    b4 = 125/594
    b5 = 0
    b6 = 512/1771
    
    bs1 = 2825/27648
    bs2 = 0
    bs3 = 18575/48384
    bs4 = 13525/55296
    bs5 = 277/14336
    bs6 = 1/4
    
    # define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    # define x at 1 step
    x_ipo = xi + h
    
    # declare a temp y array
    y_temp = np.zeros(nv)
    
    # get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    # get k2 vals
    y_derivs = dydx(xi + c2*h, yi +a21*k1)
    k2[:] = h*y_derivs[:]
    
    # get k3 vals
    y_derivs = dydx(xi+c3*h,yi+a31*k1+a32*k2)
    k3[:] = h*y_derivs[:]
    
    # get k4 vals
    y_derivs = dydx(xi + c4*h, yi+a41*k1+a42*k2+a43*k3)
    k4[:] = h*y_derivs[:]
    
    # get k5 vals
    y_derivs = dydx(xi+c5*h,yi+a51*k1+a52*k2+a53*k3+a54*k4)
    k5[:] = h*y_derivs[:]
    
    # get k6 vals
    y_derivs = dydx(xi+c6*h,yi+a61*k1+a62*k2+a63*k3+a64*k4+a65*k5)
    k6[:] = h*y_derivs[:]
    
    # get ypo vals
    yipo = yi + b1*k1 + b2*k2 + b3*k3 + b4*k4 + b5*k5 +b6*k6
    
    # get yspo vals
    yispo = yi + bs1*k1 + bs2*k2 + bs3*k3 +bs4*k4 + bs5*k5 + bs6*k6
    
    # comp yipo
    # and yipo* for error est.
    return yipo, yispo

# Define an adaptive step size driver for RK4

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    # define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    # set a max num iterations
    imax = 10000
    
    i = 0
    
    # create an error
    Delta = np.full(nv,2*tol)
    
    # remember the step
    h_step = h
    
    # adjust step
    while (Delta.max()/tol) > 1.0:

        # estimate our error by taking one step of size h 
        # vs two steps of size h/2
        
        # replace 3 lines with 1
        y_2, y_1 = ck_mv_core(dydx,x_i,y_i,nv,h_step)
#         y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
#         y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
#         y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)

        # compute an error

        Delta = np.fabs(y_2 - y_1)

        # if the error is too large, take a smaller step
        if (Delta.max()/tol) > 1.0:

            # our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)

        # check interation
        if i > imax:
            print("Too many iterations if rk4_mv_ad()")
            raise StopIteration("Ending of i  = ",i)

        i += 1
    
    # next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    # return the answer,new step, and the step we actually took
    return y_2, h_new, h_step

# Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    #dydx is the deriv wrt x
    # a is the lower bound
    # b is the upper bound
    # y_a are the boundary conditions
    # tol is the tolerance for integrating y
    
    # define out starting step
    xi = a
    yi = y_a.copy()
    
    # an initial step size == make very small
    h =1.0e-4 * (b-a)
    
    # set max iter
    imax = 10000
    
    i = 0
    
    # set num of coupled odes to the
    # size of y_a
    nv = len(y_a)
    
    # set the inital conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    # set a flag
    flag = 1
    
    # look until reach right side
    while flag:
        
        # calc y_i + 1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
                                # call correct function
        
        # update step
        h = h_new
        
        # prevent over shoot
        if xi+h_step > b:
            
            # take a smaller step
            h = b-xi
            
            #recalc y_i + 1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
                                    # call correct func
            
            # break
            flag = 0
    
        # update vals
        xi += h_step
        yi[:] = yi_new[:]

        # add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new

        # prevent too many iter
        if i > imax:
            print("Max iterations reached.")
            raise StopIteration("Iteration number = ", i)

        i += 1
    
        # output some info
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)

        # break if new xi is == b
        if xi == b:
            flag = 0
    
    return x,y

# Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

# perform the integration
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

# Plot the results

In [ ]:
plt.plot(x,y[:,0],'o',label="y(x)")
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='col(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)
plt.show()

# Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0] - sine)
dydx_error = (y[:,1] - cosine)

plt.plot(x, y_error, label='y(x) Error')
plt.plot(x, dydx_error, label='dydx(x) Error')
plt.legend(frameon=False)